# In-vivo evidence for increased tau deposition in temporal lobe epilepsy

### Content
1. Participants
1.  *Figure 1:* Mapping of tau uptake in TLE
1.  *Figure 2:* Network contextualization
1.  *Figure 3:* Relationship to clinical and cognitive variables
1. Supplementary figures


In [ ]:
# Libraries


## Participants

------------------------
## **Figure 1:** Mapping of tau uptake in TLE
### A | Mean Tau MK6240 uptake by group

### B.1 | Tau MK6240 uptake group difference

### B.2 | Mean SUVR values for significant regions in each hemisphere by group

### C.1 | Subject-Wise Distribution of Cortical MK6240 Uptake

### C.2  | Individual examples of of Cortical MK6240 Uptake

----------------
## **Figure 2:** Network contextualization
### A | Relationship Between Tau MK6240 Group Differences and Functional Connectome

### Mean functional connectome as adjacency matrix

### Mean functional connectome as network

### Tau MK6240 weighted by the functional connectome neighbours

### B | Relationship Between Tau MK6240 Group Differences and Structural Connectome

### Mean structural connectome as adjacency matrix

### Mean structural connectome as network

### Tau MK6240 weighted by the structural connectome neighbours

-----------------
## **Figure 3:** Relationship to clinical and cognitive variables

### A | Tau MK6240 SUVR and clinical relationships

### Effect of age

### Effect of duration

### B | Tau MK6240 SUVR and behavioral relationships

### Epitrack score

### Episodic memory

### Semantic memory

---------------------
# Supplementary figures

## Cortical map of abnormal MK6240 SUVR probability

## Group difference in cortical thickness

## Group difference in MK6240 controlled by thickness

## Mean longitudinal MK6240 by subject per group